In [77]:
# read each file into a pandas dataframe 
# then store all of the dataframes in a dictionary
import numpy as np
import pandas as pd
import re

data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"]
data = {}
for file_name in data_files:
    df = pd.read_csv(file_name)
    key = file_name.split('.')[0]
    data[key] = df

#### Prepare the SAT results data

In [78]:
# convert the scores from a string to numeric data type
score_cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
for col_name in score_cols:
    data['sat_results'][col_name] = pd.to_numeric(data['sat_results'][col_name], errors="coerce")

# create a sat_score column with the total score
data['sat_results']['sat_score'] = data['sat_results'][score_cols].apply(np.sum, axis=1)
data['sat_results'].head(3)

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0


#### Display the demographics data

In [79]:
data['demographics'].head(3)

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2


#### Prepare the class size data

DBN is a unique ID for each school, but, unlike the other data sets, class_size doesn't have that column. It does, however, have the CSD and SCHOOL CODE columns, which combine to make the unique DBN for each school.

In [80]:
def zfill2(integer):
    return(str(integer).zfill(2)) # left filled with ASCII '0' digits to make a string of length 2

In [81]:
col_names = data['class_size'].columns
data['class_size']['padded_csd'] = data["class_size"]["CSD"].apply(zfill2)

In [82]:
data['class_size']['DBN'] = data['class_size']['padded_csd'] + data['class_size']['SCHOOL CODE']
data['class_size'] = data['class_size'][col_names.insert(0,'DBN')]
data['class_size'].head(3)

,DBN,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,01M015,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,01M015,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,01M015,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN


#### Prepare the high school directory data

In [85]:
def extract_latitude(string):
    #input: string, which must include '(lat, long)' e.g. '(-30, 170)'
    regex = "\(.+\)" # everything between the parentheses
    coordinates = re.findall(regex, string)[0] # e.g. return '(-30, 170)'
    latitude = coordinates.split(',')[0].replace('(','') # e.g. '-30'
    return latitude

In [86]:
def extract_longitude(string):
    #input: string, which must include '(lat, long)' e.g. '(-30, 170)'
    regex = "\(.+\)" # everything between the parentheses
    coordinates = re.findall(regex, string)[0] # e.g. return '(-30, 170)'
    longitude = coordinates.split(',')[1].replace(' ','').replace(')','') # e.g. '170'
    return longitude

In [92]:
# create 'DBN' column, remove 'dbn' column
col_names = data['hs_directory'].columns.tolist()
if 'dbn' in data['hs_directory']: # the 'if' statement allows this cell to be run more than once
    data['hs_directory']['DBN'] = data['hs_directory']['dbn'] # copy 'dbn' column to 'DBN'
    col_names.insert(0,'DBN')
    col_names.remove('dbn')
    data['hs_directory'] = data['hs_directory'][col_names]

# extraxt the latitudes and longitudes from the 'Location 1' column
data['hs_directory']['lat'] = data['hs_directory']['Location 1'].apply(extract_latitude)
data['hs_directory']['lon'] = data['hs_directory']['Location 1'].apply(extract_longitude)

# convert the string latitudes and longitudes values to a numeric data type
data['hs_directory']['lat'] = pd.to_numeric(data['hs_directory']['lat'], errors='coerce')
data['hs_directory']['lon'] = pd.to_numeric(data['hs_directory']['lon'], errors='coerce')

data['hs_directory'].head(3)

,DBN,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,lat,lon
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9.0,12,NaN,NaN,...,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",9.0,35.0,213.0,3029686.0,3.011870e+09,Crown Heights South ...,40.670299,-73.961648
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9.0,12,NaN,NaN,...,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",3.0,16.0,135.0,2004526.0,2.026340e+09,Morrisania-Melrose ...,40.827603,-73.904475
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6.0,12,NaN,NaN,...,NaN,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",4.0,14.0,209.0,2008336.0,2.028590e+09,West Concourse ...,40.842414,-73.916162


#### Prepare the Survery Data

The voluntary responses in the survey data indicate how parents, teachers, and students feel about school safety, academic performance, and more.

In [88]:
# Survey data is tab-separated with Windows 1252 encoding
all_survey = pd.read_csv('survey_all.txt', delimiter="\t", encoding="windows-1252")
d75_survey = pd.read_csv('survey_d75.txt', delimiter="\t", encoding="windows-1252")

In [89]:
survey = pd.concat([all_survey, d75_survey], axis=0) # added District 75 to the survey data
survey['DBN'] = survey['dbn'] # copy the 'dbn' column and capitalize the column name
col_names = ["dbn", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
survey = survey[col_names] # keep only the columns we care about
print(survey.shape) # 1702 schools/rows
survey.head()

(1702, 23)


,dbn,rr_s,rr_t,rr_p,N_s,N_t,N_p,saf_p_11,com_p_11,eng_p_11,...,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11
0,01M015,NaN,88,60,NaN,22.0,90.0,8.5,7.6,7.5,...,7.6,7.9,NaN,NaN,NaN,NaN,8.0,7.7,7.5,7.9
1,01M019,NaN,100,60,NaN,34.0,161.0,8.4,7.6,7.6,...,8.9,9.1,NaN,NaN,NaN,NaN,8.5,8.1,8.2,8.4
2,01M020,NaN,88,73,NaN,42.0,367.0,8.9,8.3,8.3,...,6.8,7.5,NaN,NaN,NaN,NaN,8.2,7.3,7.5,8.0
3,01M034,89.0,73,50,145.0,29.0,151.0,8.8,8.2,8.0,...,6.8,7.8,6.2,5.9,6.5,7.4,7.3,6.7,7.1,7.9
4,01M063,NaN,100,60,NaN,23.0,90.0,8.7,7.9,8.1,...,7.8,8.1,NaN,NaN,NaN,NaN,8.5,7.6,7.9,8.0
